In [1]:
# !pip install -r requirements.txt

In [2]:
%matplotlib nbagg
from fastmri_recon.data.utils.masking.gen_mask_tf import gen_mask_tf
from fastmri_recon.models.utils.fourier import tf_unmasked_op as tf_fft
from fastmri_recon.models.utils.fourier import tf_unmasked_adj_op as tf_ifft
from fastmri_recon.models.utils.fourier import tf_op as tf_masked_fft
from fastmri_recon.models.utils.fourier import tf_adj_op as tf_masked_ifft
from fastmri_recon.models.utils.masking import _mask_tf
import matplotlib.pyplot as plt
import numpy as np
from skimage.data import shepp_logan_phantom
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

In [3]:
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['image.cmap'] = 'gray'

In MRI, data is acquired in the k-space. This k-space $y$ is (in an ideal version) the fourier transform $F$ of the anatomical image $x$: $y = Fx$.

However, because each point (rather each trajectory) in the k-space takes time to acquire, we are going to subsample the fourier coefficients according to a given pattern $\Omega$: $y = F_{\Omega}x$.

This is an ill-posed inverse problem.

In [4]:
phantom = shepp_logan_phantom()

AttributeError: 'NoneType' object has no attribute 'read'

In [ ]:
%debug

In [ ]:
# let's load our data and see what it looks like
im_shape = [256, 256]
# we add a batch and a channel dimension because the ops need it tow work correctly
# indeed in tf, the images are usually in the shape NHWC
image = tf.complex(tf.random.normal(im_shape), tf.random.normal(im_shape))[None, ..., None]
kspace = tf_fft(image)
fig, axs = plt.subplots(1, 2)
axs[0].imshow(np.squeeze(tf.abs(image)))
axs[0].set_title('Magnitude of the image')
axs[1].imshow(np.squeeze(tf.abs(kspace)))
axs[1].set_title('Magnitude of the fully-sampled\n k-space')

A naive solution is to take the inverse fourier transform of the sampled coefficients, with non-sampled coefficients filled to 0. The solution is then called the zero-filled solution.

In [ ]:
# we are going to accelerate the MRI acquisition by a factor 2
mask = gen_mask_tf(kspace[..., 0], accel_factor=2)
masked_kspace = _mask_tf([kspace, mask])
zero_filled_image = tf_ifft(masked_kspace)

fig, axs = plt.subplots(1, 2)
axs[0].imshow(np.squeeze(tf.abs(zero_filled_image)))
axs[0].set_title('Magnitude of the zero-filled image')
axs[1].imshow(np.squeeze(tf.abs(masked_kspace)))
axs[1].set_title('Magnitude of the undersampled\n k-space')

As you can see, the image is strongly aliased. We need a better solution.

For that, we will need to solve an optimisation problem with a data-consistency term and a regularisation term:
$$
argmin_{x \in \mathbb{C}^n} \frac12 \|F_{\Omega} x - y\|_2^2 + \lambda \|x\|_{TV} 
$$

In [ ]:
# let's write that in tf
x = tf.Variable(zero_filled_image, trainable=True, name='solution')
lamb = tf.constant(1.0)
def return_loss():
    loss = 0.5 * tf.nn.l2_loss(tf.abs(tf_masked_fft([x, mask]) - kspace)) + lamb * tf.image.total_variation(x)
    return loss
return_loss()

In [ ]:
optimizer = SGD(learning_rate=1.0)
n_iter = 100
for i_iter in range(n_iter)
    optimizer.minimize(return_loss, [x])
    return_loss()